In [1]:
%%writefile conf/config.yaml
defaults :
  - modulus_default
  - arch:
      - fully_connected
  - scheduler: tf_exponential_lr
  - optimizer: adam
  - loss: sum
  - _self_

save_filetypes : "vtk,npz"

scheduler:
  decay_rate: 0.95
  decay_steps: 100

training:
  rec_results_freq: 1000
  max_steps: 1000

batch_size:
  IC: 1000
  interior: 2000
    

Overwriting conf/config.yaml


In [9]:
%%writefile fhnModulus.py

from sympy import Symbol, Eq
from sympy import Symbol, Function, Number
from modulus.eq.pde import PDE
import numpy as np
import matplotlib.pyplot as plt
from scipy import interpolate
import modulus
from modulus.hydra import instantiate_arch, ModulusConfig
from modulus.solver import Solver
from modulus.domain import Domain
from modulus.geometry.primitives_1d import Point1D
from modulus.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseBoundaryConstraint,
)
from modulus.domain.validator import PointwiseValidator
from modulus.key import Key
from modulus.node import Node
from modulus.eq.pde import PDE
import numpy as np

from modulus.utils.io import csv_to_dict
from modulus.solver import Solver
from modulus.geometry import Parameterization
from modulus.geometry.primitives_3d import Box, Channel, Plane
from modulus.key import Key
from modulus.node import Node

from modulus.geometry import Parameterization

a=0.001
def generateExactSolution(t,dt,x0):
    
    n=int(t/dt)
    Sol=np.zeros(n)
    Sol[0]=x0
    T=0
    k=0
    while(k<n-1):
        x=Sol[k]
        Sol[k+1]=x*(x)*(1-x)*dt +  x
        T=T+dt
        k=k+1
    return Sol

class SpringMass(PDE):


    def __init__(self):

      
     

        t = Symbol("t")
        input_variables = {"t": t}

        x1 = Function("x1")(*input_variables)



        self.equations = {}
        self.equations["ode_x1"] = x1*(1-x1)*(x1) -x1.diff(t)


                                    
        
@modulus.main(config_path="conf", config_name="config")
def run(cfg: ModulusConfig) -> None:
    x0=0.7
    c1=1/x0 -1
    # make list of nodes to unroll graph on
    sm = SpringMass()
    sm_net = instantiate_arch(
        input_keys=[Key("t"),Key("K")],
        output_keys=[Key("x")],
        cfg=cfg.arch.fully_connected,
    )
    print(sm.make_nodes())
    nodes = sm.make_nodes() 

   
    # add constraints to solver
    # make geometry
    geo = Point1D(0)
    t_max = 10.0
    t_symbol = Symbol("t")
    x = Symbol("x")
    time_range = (0, t_max)

    K_max = 10.0
    K_symbol = Symbol("K")
    K_range =(0, K_max)

    param_ranges = {
    K_symbol: K_range,
    t_symbol:time_range
}
    pr=Parameterization(param_ranges)
    # make domain
    domain = Domain()

    # initial conditions
    IC = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"x": x0},
        batch_size=cfg.batch_size.IC,
        lambda_weighting={
            "x": 1.0,


        },
        parameterization={t_symbol: 0},
    )
    domain.add_constraint(IC, name="IC")

    
    
    # solve over given time period
    interior = PointwiseBoundaryConstraint(
        nodes=nodes,
        geometry=geo,
        outvar={"ode_x": 0.0},
        batch_size=cfg.batch_size.interior,
        parameterization=pr,
    )
    domain.add_constraint(interior, "interior")

    # add validation data
    deltaT = 0.001
    t = np.arange(0, t_max, deltaT)
    t = np.expand_dims(t, axis=-1)
    K = np.arange(0, t_max, deltaT)
    K = np.expand_dims(t, axis=-1)
    Sol=generateExactSolution(t_max,deltaT,x0)
    sol = interpolate.interp1d([deltaT*i for i in range( int(t_max/deltaT))],Sol) 
    
    invar_numpy = {"t": t,"K":K}
    outvar_numpy = {
        "x": Sol
    }
    validator = PointwiseValidator(
        nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy, batch_size=1024
    )
    domain.add_validator(validator)

    # make solver
    slv = Solver(cfg, domain)

    # start solver
    slv.solve()


if __name__ == "__main__":
    run()


Overwriting fhnModulus.py


In [10]:
!rm -r outputs || true ##se não limpar o output ele aproveita o treinamento, mesmo se mudar o modelo
!python fhnModulus.py

[02:49:15] - JIT using the NVFuser TorchScript backend
[02:49:15] - JitManager: {'_enabled': True, '_arch_mode': <JitArchMode.ONLY_ACTIVATION: 1>, '_use_nvfuser': True, '_autograd_nodes': False}
[02:49:15] - GraphManager: {'_func_arch': False, '_debug': False, '_func_arch_allow_partial_hessian': True}
Error executing job with overrides: []
Traceback (most recent call last):
  File "/home/jovyan/t/fhnModulus.py", line 101, in run
    IC = PointwiseBoundaryConstraint(
  File "/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/modulus-22.9-py3.9.egg/modulus/domain/constraint/continuous.py", line 287, in __init__
    lambda_weighting = _compute_lambda_weighting(
  File "/opt/saturncloud/envs/saturn/lib/python3.9/site-packages/modulus-22.9-py3.9.egg/modulus/domain/constraint/utils.py", line 20, in _compute_lambda_weighting
    lambda_weighting[key] = np_lambdify(
TypeError: modulus.utils.sympy.numpy_printer.np_lambdify.<locals>.loop_grouped_lambda.<locals>.grouped_lambda() got multipl

In [4]:
import numpy as np
import matplotlib.pyplot as plt

base_dir = "outputs/fhnModulus/validators/"

# plot in 1d
data = np.load(base_dir + "validator.npz", allow_pickle=True)
data = np.atleast_1d(data.f.arr_0)[0]

plt.plot(data["t"], data["true_x1"], label="True x1")

plt.plot(data["t"], data["pred_x1"], label="Pred x1")

plt.legend()
plt.savefig("comparison.png")



FileNotFoundError: [Errno 2] No such file or directory: 'outputs/fhnModulus/validators/validator.npz'